Make MTSv3 data in the same data format as CTW1500. <br>
Use detection only mode of MTSv3 -> use "dummy" label for word

In [1]:
import xml.etree.ElementTree as ET
import os
import shutil
from natsort import natsorted

for data_split in ["train", "val", "test"]:
    print(data_split)
    with open(f'./books_{data_split}.txt', 'r') as book_list:
        manga_list = book_list.readlines()
    
    os.makedirs(f"./MTSv3_data/{data_split}_images/", exist_ok=True)
    os.makedirs(f"./MTSv3_data/{data_split}_gts/", exist_ok=True)
    
    for manga_name in natsorted(manga_list):
        manga_name = manga_name.strip()
        
        tree = ET.parse(f'./annotations/{manga_name}.xml')
        root = tree.getroot()
        
        for page in root.iter('page'):
            page_index = page.attrib["index"]
            new_tree = ''
            new_element = ET.Element('Annotations')
            with open(f'./MTSv3_data/{data_split}_gts/{manga_name}_{page_index}.jpg.txt', 'w', encoding="utf-8") as file:
                for onomatopoeia in page.iter('onomatopoeia'):
                    attrib = onomatopoeia.attrib 

                    x_list = []
                    y_list = []
                    for key, val in attrib.items():
                        if key == 'id': 
                            continue

                        elif 'x' in key:
                            x_list.append(int(val))

                        elif 'y' in key:
                            y_list.append(int(val))

                    if len(x_list) == 2: # rectangle 일때.
                        x_list = [x_list[0], x_list[1], x_list[1], x_list[0]]
                        y_list = [y_list[0], y_list[0], y_list[1], y_list[1]]

                    point_list = []
                    for x, y in zip(x_list, y_list):
                        point_list.append(str(x))
                        point_list.append(str(y))
                    # print(point_list)

                    label = "dummy"
                    point_list = ",".join(point_list)
                    file.write(f"{point_list},{label}\n")

            page_index_zfill = page_index.zfill(3)
            shutil.copyfile(f"./images/{manga_name}/{page_index_zfill}.jpg", f"./MTSv3_data/{data_split}_images/{manga_name}_{page_index}.jpg")

train
val
test


In [2]:
"""To train with bbox annotation. to compare bbox with polygon annotation"""
import xml.etree.ElementTree as ET
import os
import shutil
from natsort import natsorted

for data_split in ["train"]:
    print(data_split)
    with open(f'./books_{data_split}.txt', 'r') as book_list:
        manga_list = book_list.readlines()
    
    os.makedirs(f"./MTSv3_data/{data_split}_images_bbox/", exist_ok=True)
    os.makedirs(f"./MTSv3_data/{data_split}_gts_bbox/", exist_ok=True)
    
    for manga_name in natsorted(manga_list):
        manga_name = manga_name.strip()
        
        tree = ET.parse(f'./annotations/{manga_name}.xml')
        root = tree.getroot()
        
        for page in root.iter('page'):
            page_index = page.attrib["index"]
            new_tree = ''
            new_element = ET.Element('Annotations')
            with open(f'./MTSv3_data/{data_split}_gts_bbox/{manga_name}_{page_index}.jpg.txt', 'w', encoding="utf-8") as file:
                for onomatopoeia in page.iter('onomatopoeia'):
                    attrib = onomatopoeia.attrib 

                    x_list = []
                    y_list = []
                    for key, val in attrib.items():
                        if key == 'id': 
                            continue

                        elif 'x' in key:
                            x_list.append(int(val))

                        elif 'y' in key:
                            y_list.append(int(val))

                    if len(x_list) == 2: # rectangle 일때.
                        x_list = [x_list[0], x_list[1], x_list[1], x_list[0]]
                        y_list = [y_list[0], y_list[0], y_list[1], y_list[1]]
                    
                    xmin = str(min(x_list))
                    ymin = str(min(y_list))
                    xmax = str(max(x_list))
                    ymax = str(max(y_list))
                    # bbox!
                    point_list = [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]
                    
                    label = "dummy"
                    point_list = ",".join(point_list)
                    file.write(f"{point_list},{label}\n")

            page_index_zfill = page_index.zfill(3)
            shutil.copyfile(f"./images/{manga_name}/{page_index_zfill}.jpg", f"./MTSv3_data/{data_split}_images_bbox/{manga_name}_{page_index}.jpg")

train
